# Підготовка даних

## Файл csv із збігами та їх прізвищами

Мають бути такі колонки

- surname - прізвище предка збіженця. У чоловічому роді, українською мовою
- name - ім'я збіженця
- cm - число, спільна днк в сантиморганах
- url - покликання на збіженця

# Параметри

In [66]:
# Всі необхідні файли потрібно покласи у цей каталог
directory = "data"

# Файл із збіженцями та прізвищами їх предків
filename = "DNA Genealogy - Surnames_my"

limit_counties = 15

# Відкриваємо вхідний файл із збіженцями прізвищами їх предків та базу даних

In [67]:
import sqlite3

db = sqlite3.connect(f'{directory}/surnames.sqlite3')

In [68]:
import pandas as pd
matches_df = pd.read_csv(f'{directory}/{filename}.csv')
matches_df

,surname,name,cm,url
0,Гоменко,Henri Louis Helbach,43.8,https://www.myheritage.com.ua/dna/match/D-FE4A...
1,Шиліпук,Olya Shylipuk,41.7,https://www.myheritage.com.ua/dna/match/D-FE4A...
2,Бурчак,Olya Shylipuk,41.7,https://www.myheritage.com.ua/dna/match/D-FE4A...
3,Савва,Adena Savva,41.4,https://www.myheritage.com.ua/dna/match/D-F513...
4,Шмаленко,Inna Shmalenko,40.5,https://www.myheritage.com.ua/dna/match/D-FE4A...
...,...,...,...,...
257,Кендзерський,Karli Maruschak,19.9,https://www.myheritage.com.ua/dna/match/D-FE4A...
258,Марущак,Karli Maruschak,19.9,https://www.myheritage.com.ua/dna/match/D-FE4A...
259,Пелай,Karli Maruschak,19.9,https://www.myheritage.com.ua/dna/match/D-FE4A...
260,Шушкевич,Karli Maruschak,19.9,https://www.myheritage.com.ua/dna/match/D-FE4A...


# Підготовка таблиць

In [69]:
table_distribution = "distribution"

cursor = db.cursor()


In [70]:
cursor.execute(f'DROP TABLE IF EXISTS {table_distribution}')
cursor.execute(
    f'''
    CREATE TABLE {table_distribution} (
    	"county"	INT NOT NULL,
        "surname"   TEXT NOT NULL,
        "cnt" INT NOT NULL,
        "score"	    REAL NOT NULL
    )
    '''
)

In [71]:
db.commit()

# Обробка прізвищ

In [72]:
for idx, row in matches_df.iterrows():
    # print(row)
    surname = row['surname'].upper()
    cm = row['cm']
    match = row['name']
    print(match)

    sql = f'''
        INSERT INTO {table_distribution}
        SELECT
        	P."Районнародження" AS county,
            "{surname}",
        	COUNT(*) as cnt,
        	{cm}
        FROM People P
        WHERE P."Прізвищє" = "{surname}"
        AND P."Стать" = 1
        AND county IS NOT NULL
        GROUP BY county
        ORDER BY cnt DESC
        LIMIT 15
    '''

    # print(sql)

    cursor.execute(sql)
    # break
    
db.commit()

print("All records processed")
    

Henri Louis Helbach
Olya Shylipuk
Olya Shylipuk
Adena Savva
Inna Shmalenko
Inna Shmalenko
Inna Shmalenko
Aleksander Tsarkov
Марія Платонівна Боднар
Марія Платонівна Боднар
Oleksandr Martynienko
Anna Bruzgo
Александр Поликаренко
Александр Поликаренко
Оксана Побережник
Оксана Побережник
Андрей Ваврыш
Андрей Ваврыш
Андрей Ваврыш
Uladzislau Luchkouski
Uladzislau Luchkouski
Tetiana Shvorobei
Tetiana Shvorobei
Lukasz Nikoniuk
Lukasz Nikoniuk
Lukasz Nikoniuk
Lukasz Nikoniuk
Oleg Kolomeets
Henryk (Piotr) Cieszko
Henryk (Piotr) Cieszko
Olga Saleh
Salem Sharp
Сергій Дмитрович Хватков
Сергій Дмитрович Хватков
Сергій Дмитрович Хватков
Kelly Marceniuk
Kelly Marceniuk
Karyna Reida
Maret Iarmak
Maret Iarmak
MIKE WUS
MIKE WUS
Elena Sozinova
Тетяна Челяда
Тетяна Челяда
Terry Kostyna
Terry Kostyna
Юра Федорчук
Anna Tovstukha
Kozový Daniel
Kozový Daniel
Дмитрий Порайко
Vadym Kovadlo
Vadym Kovadlo
Vadym Kovadlo
Volodymyr Turiy
Евгений Митрофанов
Paul Donald Bodnarchuk
Paul Donald Bodnarchuk
Василь Капацин

# Створимо результуючий документ

In [73]:
import xlsxwriter

In [74]:
import os, errno

resfile = f'{directory}/{filename}.xlsx'
try:
    os.remove(filename)
except OSError:
    pass
workbook = xlsxwriter.Workbook(resfile)

In [75]:
def add_row(ws, rn, content, format = None):
    for i, c in enumerate(content):
        # print(f"Write {c} to ({i},{rn})")
        ws.write(rn, i, c, format)

In [76]:
header_format = workbook.add_format()
header_format.set_bold()

def add_sheet_sql(name, header, sql, width = None):
    worksheet = workbook.add_worksheet(name)
    # if width is not None:
    #     for w in width:
            
    add_row(worksheet, 0, header, header_format)
    cursor.execute(sql)
    for i, row in enumerate(cursor.fetchall()):
        add_row(worksheet, i+1, row)

    worksheet.autofit()

In [77]:
add_sheet_sql("Райони", ["Область", "Район", "Ранг"], 
              '''
            SELECT
            	IFNULL(r.name, C.region) as region,
                IFNULL(c.name, D.county) as county,
            	SUM(score) as score
            FROM
            	distribution D
            LEFT JOIN counties C ON D.county = C.id
            LEFT JOIN regions R ON c.region = R.id
            GROUP BY D.county
            ORDER BY score DESC
              '''
             )

In [78]:
add_sheet_sql("Прізвища", ["Прізвище", "Область", "Район", "Кількість", "Ранг"], 
              f'''
            SELECT 
                surname,
                IFNULL(r.name, c.region) as region,
                IFNULL(c.name, d.county) as county,
                cnt,
                score
            FROM {table_distribution} d
            LEFT JOIN counties as c ON d.county = c.id
            LEFT JOIN regions as r on c.region = r.id
            order by d.surname    
              '''
             )

In [79]:
workbook.close()

# Закрити базу даних

In [80]:
db.close()